<a href="https://colab.research.google.com/github/Jahan08/Amber-tutorial/blob/main/hERG-Retrieve-Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install chembl_webresource_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [4]:
target = new_client.target
target_query = target.search('hERG')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'Q12809', 'xref_name': None, 'xre...",Homo sapiens,HERG,19.0,False,CHEMBL240,"[{'accession': 'Q12809', 'component_descriptio...",SINGLE PROTEIN,9606
1,[],Homo sapiens,Melanin-concentrating hormone receptor 2/HERG,12.0,False,CHEMBL4106188,"[{'accession': 'Q12809', 'component_descriptio...",SELECTIVITY GROUP,9606
2,[],Homo sapiens,Voltage-gated potassium channel,3.0,False,CHEMBL2362996,"[{'accession': 'P51787', 'component_descriptio...",PROTEIN FAMILY,9606


In [5]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL240'

In [6]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [7]:
df = pd.DataFrame.from_dict(res)
df

KeyboardInterrupt: ignored

In [ ]:

# considering nM unit for MIC
df = df[(df['standard_units'] == 'nM')]
df = df[(df['relation'] == '=')]
len(df)

9136

In [ ]:
# considering nM unit for MIC
df = df[(df['standard_units'] == '%')]
df = df[(df['relation'] == '=')]
len(df)

In [8]:
df = pd.read_csv('/content/hERG_IC50_nM.csv', header = 0)

In [9]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i > 30000:
        STATUS.append("Yes") #active
        
    elif i < 1000:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('standard_value', 1)
df['value'] = fix
df['Activity'] = STATUS
df.head(3)

<ipython-input-9-1694ea1f6290>:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('standard_value', 1)


,assay_chembl_id,assay_description,canonical_smiles,Source,Name,hERG_uM,Activity,value
0,CHEMBL841079,Inhibition of hERG currents Kv11.1,O=C1NCCN1CCN1CCC(c2cn(-c3ccc(F)cc3)c3ccc(Cl)cc...,J Med Chem,CHEMBL12713,0.0140,No,14.0
1,CHEMBL691014,K+ channel blocking activity in human embryoni...,O=C(CCCN1CC=C(n2c(=O)[nH]c3ccccc32)CC1)c1ccc(F...,J Med Chem,CHEMBL1108,0.0322,No,32.2
2,CHEMBL691013,K+ channel blocking activity in human embryoni...,O=C(O[C@@H]1C[C@@H]2C[C@H]3C[C@H](C1)N2CC3=O)c...,J Med Chem,CHEMBL2368925,5.9500,IN,5950.0


In [ ]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i > 80:
        STATUS.append("Yes") #active
        
    elif i == 0:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df.head(3)

In [10]:
# considering nM unit for MIC
df = df[(df['Activity'] != 'IN')]
len(df)

2968

In [11]:

df.to_csv('hERG_IC50_nM_new.csv', index=False)
     